# Population Forecast
1. Find the population growth rates of greater Melbourne and rest of vic in 2022 to 2025.
2. Find out whether the suburb belongs to Greater Melbourne or REST OF VIC.
3. Find the suburb population in 2022 to 2025 by the population in  2021 and the growth rate.

In [40]:
# Import packages
import pandas as pd
import csv
import geopandas as gpd

Read the data

In [41]:
population = pd.read_csv('../data/raw/population_for_suburb.csv',index_col=0)
population_pred_greater_melbourne= pd.read_csv('../data/raw/external_data/population_pred_greater_melbourne.csv',index_col=0)
population_pred_rest_vic= pd.read_csv('../data/raw/external_data/population_pred_rest_vic.csv',index_col=0)
properties_preporcessed_1=pd.read_csv('../data/raw/properties_preporcessed_1.csv',index_col=0)
SA2_2016=gpd.read_file('../data/raw/external_data/SA2_shape_2016/SA2_2016_AUST.shp')
SA2_2016=SA2_2016[SA2_2016['STE_NAME16'] == 'Victoria']


Find the population growth rates

In [42]:
# The population growth rates of greater Melbourne in 2022 to 2025.
population_pred_greater_melbourne=population_pred_greater_melbourne.loc[2022:2025,['Annual population growth(e)']]/100 + 1

# The population growth rates of rest of vic in 2022 to 2025.
population_pred_rest_vic=population_pred_rest_vic.loc[2022:2025,['Annual population growth(e)']]/100 + 1

In [43]:
population_pred_greater_melbourne

,Annual population growth(e)
2022,1.023
2023,1.022
2024,1.021
2025,1.020


In [44]:
population_pred_rest_vic

,Annual population growth(e)
2022,1.011
2023,1.010
2024,1.010
2025,1.009


Suburb correspondence of SA2 and suburb.

In [45]:
# Correspondence between Sa2 Code in 2016 and Locality Code.
properties_preporcessed_1=properties_preporcessed_1[['SA2_MAIN16','LOC_PID']].copy()
properties_preporcessed_1=properties_preporcessed_1.groupby(['SA2_MAIN16','LOC_PID']).count()
properties_preporcessed_1=properties_preporcessed_1.reset_index()
properties_preporcessed_1.SA2_MAIN16=properties_preporcessed_1.SA2_MAIN16.astype('int')
properties_preporcessed_1

,SA2_MAIN16,LOC_PID
0,201011001,loc82f5b45a790d
1,201011001,loc8c852c59ffe5
2,201011001,loc98325a7e67bf
3,201011002,loc2c1864e3cbb6
4,201011002,loc76166a14c514
...,...,...
840,217041478,loce829f646f7ca
841,217041478,locf00a01fc543b
842,217041479,loc1f73672977ce
843,217041480,loc1f73672977ce


Find out whether the suburb belongs to Greater Melbourne or REST OF VIC.

In [46]:
# Correspondence between SA2 and regions, the region is Greater Melbourne or REST OF VIC.
SA2_2016 = SA2_2016[['SA2_MAIN16','GCC_NAME16']]
SA2_2016 = SA2_2016[SA2_2016['GCC_NAME16'].isin(['Greater Melbourne','Rest of Vic.'])].copy()
SA2_2016 = SA2_2016.set_index('SA2_MAIN16').reset_index()
SA2_2016.SA2_MAIN16 = SA2_2016.SA2_MAIN16.astype('int')
SA2_2016

,SA2_MAIN16,GCC_NAME16
0,201011001,Rest of Vic.
1,201011002,Rest of Vic.
2,201011003,Rest of Vic.
3,201011004,Rest of Vic.
4,201011005,Rest of Vic.
...,...,...
457,217031476,Rest of Vic.
458,217041477,Rest of Vic.
459,217041478,Rest of Vic.
460,217041479,Rest of Vic.


In [47]:
# Correspondence between locality and regions, the region is Greater Melbourne or REST OF VIC.
df2=pd.merge(properties_preporcessed_1,SA2_2016)
df2=df2[['LOC_PID','GCC_NAME16']].copy()
df2

,LOC_PID,GCC_NAME16
0,loc82f5b45a790d,Rest of Vic.
1,loc8c852c59ffe5,Rest of Vic.
2,loc98325a7e67bf,Rest of Vic.
3,loc2c1864e3cbb6,Rest of Vic.
4,loc76166a14c514,Rest of Vic.
...,...,...
840,loce829f646f7ca,Rest of Vic.
841,locf00a01fc543b,Rest of Vic.
842,loc1f73672977ce,Rest of Vic.
843,loc1f73672977ce,Rest of Vic.


Calculate suburb population in 2022 to 2025.

In [48]:
# The locality population in 2021.
population=pd.merge(population,df2,)[['LOC_PID','suburb_population','GCC_NAME16']]

# The locality population belongs to Greater Melbourne in 2021.
Greater_Melbourne=population[population['GCC_NAME16'].isin(['Greater Melbourne'])].copy()
# The locality population belongs to Rest Of VIC in 2021.
Rest_of_Vic=population[population['GCC_NAME16'].isin(['Rest of Vic.'])].copy()

population

,LOC_PID,suburb_population,GCC_NAME16
0,loc0067a4549ed1,4897.000000,Rest of Vic.
1,loc00a9769647d7,26158.000000,Greater Melbourne
2,loc00d1503504f1,41928.000000,Greater Melbourne
3,loc00d1503504f1,41928.000000,Greater Melbourne
4,loc00e6e39d335b,1057.555556,Rest of Vic.
...,...,...,...
840,locff6258c8ea42,9003.705882,Greater Melbourne
841,locff62fb6a898a,22420.000000,Greater Melbourne
842,locffa1c8993b70,5488.609375,Greater Melbourne
843,locffb43e78ab10,2133.333333,Rest of Vic.


In [49]:
# Calculate the locality population in 2022 to 2025 in Greater Melbourne based on the population in 2021 and the growth rate.
Greater_Melbourne['2022']=Greater_Melbourne['suburb_population']*population_pred_greater_melbourne.loc[2022]['Annual population growth(e)']
Greater_Melbourne['2023']=Greater_Melbourne['2022']*population_pred_greater_melbourne.loc[2023]['Annual population growth(e)']
Greater_Melbourne['2024']=Greater_Melbourne['2023']*population_pred_greater_melbourne.loc[2024]['Annual population growth(e)']
Greater_Melbourne['2025']=Greater_Melbourne['2024']*population_pred_greater_melbourne.loc[2025]['Annual population growth(e)']
Greater_Melbourne

,LOC_PID,suburb_population,GCC_NAME16,2022,2023,2024,2025
1,loc00a9769647d7,26158.000000,Greater Melbourne,26759.634000,27348.345948,27922.661213,28481.114437
2,loc00d1503504f1,41928.000000,Greater Melbourne,42892.344000,43835.975568,44756.531055,45651.661676
3,loc00d1503504f1,41928.000000,Greater Melbourne,42892.344000,43835.975568,44756.531055,45651.661676
5,loc00f0949ea0ad,39266.294118,Greater Melbourne,40169.418882,41053.146098,41915.262166,42753.567409
6,loc00f0949ea0ad,39266.294118,Greater Melbourne,40169.418882,41053.146098,41915.262166,42753.567409
...,...,...,...,...,...,...,...
838,locff0c4d75067f,8020.000000,Greater Melbourne,8204.460000,8384.958120,8561.042241,8732.263085
840,locff6258c8ea42,9003.705882,Greater Melbourne,9210.791118,9413.428522,9611.110521,9803.332732
841,locff62fb6a898a,22420.000000,Greater Melbourne,22935.660000,23440.244520,23932.489655,24411.139448
842,locffa1c8993b70,5488.609375,Greater Melbourne,5614.847391,5738.374033,5858.879888,5976.057486


In [50]:
# Calculate the locality population in 2022 to 2025 in Rest of VIC based on the population in 2021 and the growth rate.
Rest_of_Vic['2022']=Rest_of_Vic['suburb_population']*population_pred_rest_vic.loc[2022]['Annual population growth(e)']
Rest_of_Vic['2023']=Rest_of_Vic['2022']*population_pred_rest_vic.loc[2023]['Annual population growth(e)']
Rest_of_Vic['2024']=Rest_of_Vic['2023']*population_pred_rest_vic.loc[2024]['Annual population growth(e)']
Rest_of_Vic['2025']=Rest_of_Vic['2024']*population_pred_rest_vic.loc[2025]['Annual population growth(e)']
Rest_of_Vic

,LOC_PID,suburb_population,GCC_NAME16,2022,2023,2024,2025
0,loc0067a4549ed1,4897.000000,Rest of Vic.,4950.867000,5000.375670,5050.379427,5095.832842
4,loc00e6e39d335b,1057.555556,Rest of Vic.,1069.188667,1079.880553,1090.679359,1100.495473
7,loc0115c25c528c,1106.000000,Rest of Vic.,1118.166000,1129.347660,1140.641137,1150.906907
9,loc01636b23f63e,1046.083333,Rest of Vic.,1057.590250,1068.166152,1078.847814,1088.557444
10,loc01699d7adbaa,1793.333333,Rest of Vic.,1813.060000,1831.190600,1849.502506,1866.148029
...,...,...,...,...,...,...,...
821,locfa471b99ea58,1680.000000,Rest of Vic.,1698.480000,1715.464800,1732.619448,1748.213023
831,locfdb55e6f8522,2227.111111,Rest of Vic.,2251.609333,2274.125427,2296.866681,2317.538481
833,locfe8a70072c86,3830.000000,Rest of Vic.,3872.130000,3910.851300,3949.959813,3985.509451
839,locff58d0167065,10487.000000,Rest of Vic.,10602.357000,10708.380570,10815.464376,10912.803555


In [51]:
# Concat the Greater Melbourne and REST of VIC population data.
pred = pd.concat([Greater_Melbourne,Rest_of_Vic]).drop_duplicates().drop(columns='GCC_NAME16', axis=1)
for year in ['2022', '2023', '2024', '2025']:
  pred[year] = pred[year].map(lambda x : round(x))

# Rename it
pred = pred.rename(columns={'2022' : 'population_proj_2022',
                            '2023' : 'population_proj_2023',
                            '2024' : 'population_proj_2024', 
                            '2025' : 'population_proj_2025'})
pred

,LOC_PID,suburb_population,population_proj_2022,population_proj_2023,population_proj_2024,population_proj_2025
1,loc00a9769647d7,26158.000000,26760,27348,27923,28481
2,loc00d1503504f1,41928.000000,42892,43836,44757,45652
5,loc00f0949ea0ad,39266.294118,40169,41053,41915,42754
8,loc01478eda4021,9284.850000,9498,9707,9911,10109
11,loc01c2784fcea2,4395.530303,4497,4596,4692,4786
...,...,...,...,...,...,...
821,locfa471b99ea58,1680.000000,1698,1715,1733,1748
831,locfdb55e6f8522,2227.111111,2252,2274,2297,2318
833,locfe8a70072c86,3830.000000,3872,3911,3950,3986
839,locff58d0167065,10487.000000,10602,10708,10815,10913


In [52]:
pred.to_csv("../data/raw/population_projection.csv")